# <font style="color: rgb(50,120, 229);"> Optimización con Descenso del Gradiente </font>

En este cuaderno, utilizaremos un ejemplo simple para demostrar un algoritmo llamado Descenso del Gradiente. 

El descenso del gradiente es un algoritmo de optimización basado en el gradiente que se utiliza ampliamente en el aprendizaje automático y el aprendizaje profundo para minimizar una función de pérdida ajustando iterativamente los parámetros del modelo en la dirección del descenso más pronunciado basado en el gradiente negativo.


Específicamente, veremos cómo ajustar una línea recta a través de un conjunto de puntos para determinar la pendiente de la línea. 

Para hacer esto, definiremos una **función de pérdida** que cuantifique el error entre los datos y el **modelo matemático** que elegimos para representar los datos, y usaremos esta función de pérdida para desarrollar una regla de **actualización** que convergerá iterativamente al valor óptimo. 

Concluiremos el cuaderno con una variación del algoritmo de Descenso del Gradiente llamada Descenso del Gradiente Estocástico en Mini-Batches, que es la base para el entrenamiento de redes neuronales.


<img src="./images/c3_w1_gradient_descent_demo.gif" width="700"  />

## <font style="color: rgb(50,120, 229);">  ¿Qué es optimización? </font>

La optimización es el proceso de encontrar el mejor resultado posible bajo ciertas circunstancias. 

En el aprendizaje automático, la optimización se refiere a la tarea de ajustar los parámetros de un modelo para minimizar una función de pérdida.

## <font style="color: rgb(50,120, 229);">  ¿Qué es el gradiente? </font>

El gradiente es un vector que apunta en la dirección de un máximo local de una función y su magnitud indica la tasa de cambio de la función en esa dirección.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
plt.rcParams["figure.figsize"] = (15, 6)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14

## <font style="color: rgb(50,120, 229);"> 1. Crear un conjunto de datos </font>

In [ ]:
import numpy as np

def create_data():
    # Seed manual para consistencia.
    np.random.seed(42)

    num_data = 30

    # Crear datos que son aproximadamente lineales (pero no exactamente).
    x = 10 * np.random.uniform(size=num_data)
    y = x + np.random.normal(scale=0.3, size=num_data)

    return x, y


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Crear algunos datos.
x, y = create_data()

# Generar los datos para la línea inicial con una pendiente de 2.
xmin = np.min(x)
xmax = np.max(x)

xplot = np.linspace(xmin, xmax, 2)
m0 = 2
yplot = m0 * xplot

# Graficar los datos de muestra y la suposición inicial para una línea.
plt.figure()
plt.scatter(x, y, color='blue', s=20)
plt.xlabel('x')
plt.ylabel('y')
plt.plot(xplot, yplot, 'c--')
plt.title('Datos de Muestra con Línea Inicial')
plt.text(1, 7, 'Pendiente Inicial de la Línea: ' + str(m0), fontsize=14)
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.show()

### <font style="color: rgb(50,120, 229);"> 2. Definir el modelo </font>

Nuestro modelo para los datos es una línea recta, y simplificaremos el problema para que la línea pase por el origen. 

La ecuación para tal línea es:

$$y = mx$$

El modelo tiene un único parámetro desconocido `m` (la pendiente de la línea) que deseamos calcular.


### <font style="color: rgb(50,120, 229);"> 3. Definir la función de pérdida </font>

Ahora definamos una función de pérdida que cuantifique el error entre nuestro modelo y cualquier punto de datos en particular. Para cualquier valor dado de `xi` en nuestro conjunto de datos, tenemos el valor correspondiente para `yi` así como una estimación dada por `mxi`. Entonces, tendremos un error o un residuo dado por:

$$
\text{error} = y_i - m x_i

Queremos encontrar un valor de `m` que minimice el error anterior. Los valores positivos o negativos del error son igualmente malos para nosotros. Entonces, si elevamos al cuadrado el error, podemos definir una métrica de pérdida que mide igualmente los errores en cualquier dirección (por encima o por debajo de la línea).

La línea que mejor se ajusta al conjunto de datos en su conjunto minimizaría la pérdida total en todo el conjunto de datos, por lo que queremos sumar los errores para cada punto en el conjunto de datos. En otras palabras, queremos minimizar la siguiente ecuación:

$$
\text{loss} = \sum_{i=1}^{n} (y_i - m x_i)^2
$$

Esto se conoce como la función de pérdida de suma de errores al cuadrado. Si calculamos la pérdida cuadrada promedio en todo el conjunto de datos, entonces llegamos a la función de pérdida del error cuadrático medio (MSE) que se muestra a continuación:

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - m x_i)^2
$$

Otra variación para la pérdida es el error medio absoluto (MAE) que se calcula de la siguiente manera:

$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} |y_i - m x_i|
$$

**TIP:**

Una diferencia clave entre MSE y MAE es que la función de pérdida MSE es más sensible a los valores atípicos en el conjunto de datos. 

Si deseas minimizar el efecto de los valores atípicos en los datos, entonces MAE suele ser una mejor opción para una función de pérdida porque los errores no están al cuadrado como lo estarían con MSE. 

En el resto de este cuaderno, utilizaremos la función de pérdida MSE para demostrar el descenso del gradiente. Hay otros tipos de funciones de pérdida sobre los que aprenderemos más adelante en el curso.


## <font style="color: rgb(50,120, 229);"> 4. Optimización </font>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_linear_model(x, y, m_best, xlim=(0, 10), ylim=(0, 10)):
    # Generar la línea basada en la pendiente óptima.
    xmin = np.min(x)
    xmax = np.max(x)
    ymin = np.min(y)
    ymax = np.max(y)

    xplot = np.linspace(xmin, xmax, 2)
    yplot = m_best * xplot

    # Graficar los datos y el modelo.
    plt.figure()
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.plot(xplot, yplot, 'c-')
    plt.scatter(x, y, color='blue', s=20)
    plt.xlabel('x')
    plt.ylabel('y')
    xc = .05 * (xmax - xmin)
    yc = .95 * (ymax - ymin)
    plt.text(xc, yc, 'Pendiente: ' + str(round(m_best, 3)), fontsize=14)
    plt.show()

#### <font style="color: rgb(50,120, 229);"> 4.1 Descenso del Gradiente </font>

Ahora discutamos cómo funciona el descenso del gradiente. Para un valor dado de `m`, podemos calcular el gradiente de la función de pérdida y usar ese valor para informarnos cómo ajustar `m`. Si el gradiente es positivo, entonces necesitaremos disminuir el valor de `m` para acercarnos al mínimo, y si el gradiente es negativo, necesitaremos aumentar el valor de `m`. Esta idea simple se llama Descenso del Gradiente.

Suponiendo que la función de pérdida es convexa y diferenciable, podemos calcular el gradiente de la función de pérdida con respecto a `m` en cualquier punto para lograr esto.

$$
\frac{\partial \text{MSE}}{\partial m} = -\frac{2}{n} \sum_{i=1}^{n} x_i(y_i - m x_i)
$$

Observa que estamos calculando el gradiente para cada punto en el conjunto de datos, por eso esta técnica también se conoce como **Descenso del Gradiente en Lote**, ya que estamos procesando un "lote" de datos.


Ahora podemos usar el gradiente para desarrollar una regla de actualización para `m`. Para seguir la pendiente de la curva hacia el mínimo, necesitamos mover `m` en la dirección del gradiente negativo. Sin embargo, necesitamos controlar la velocidad a la que nos movemos a lo largo de la curva para no sobrepasar el mínimo.

Por lo tanto, usamos un parámetro, `λ`, llamado tasa de aprendizaje. Este es un parámetro que requiere ajuste dependiendo del problema en cuestión.

$$
m_k = m_{k - 1} - \lambda \frac{\partial \text{MSE}}{\partial m}
$$

In [ ]:
# Configuración de parámetros.
num_iter0 = 50
lr0 = 0.005

# Valor inicial de la pendiente.
m0 = 2

max_loss = 30. # Valor arbitrario para la pérdida máxima (solo para la visualización).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_iter = num_iter0
lr = lr0
m = m0

# Inicializar el array para la pérdida en cada iteración.
loss_gd = np.zeros(num_iter)

# Calcular la pérdida.
for i in range(num_iter):
    # Calcular el gradiente usando todo el conjunto de datos.
    g = -2 * np.sum(x * (y - m * x)) / len(x)

    # Actualizar el parámetro, m.
    m = m - lr * g

    # Calcular la pérdida para el valor actualizado de m.
    e = y - m * x
    loss_gd[i] = np.sum(e * e) / len(x)

m_best = m

print('Minimum loss:   ', loss_gd[-1])
print('Best parameter: ', m_best)

# Graficar la pérdida vs iteraciones.
plt.figure()
plt.plot(loss_gd, 'c-')
plt.xlim(0, num_iter)
plt.ylim(0, np.max(loss_gd))
plt.ylabel('Loss')
plt.xlabel('Iterations')
plt.title('Gradient Descent')
plt.show()

plot_linear_model(x, y, m_best)


#### <font style="color: rgb(50,120, 229);"> 4.2 Descenso del Gradiente Estocástico </font>

En este ejemplo, solo tenemos un puñado de puntos de datos. En el mundo real, podemos tener **millones de ejemplos**. Calcular el gradiente basado en todos los puntos de datos puede ser computacionalmente costoso. Afortunadamente, usar todos los puntos de datos para calcular el gradiente es innecesario.

Podemos usar **un solo punto** de datos elegido al azar para calcular el gradiente en cada iteración. Aunque el gradiente en cada paso no es tan preciso, la idea sigue funcionando. La convergencia podría ser más lenta utilizando esta técnica porque el gradiente no es tan preciso. En la próxima sección, ampliaremos esta idea para usar un pequeño porcentaje de los datos para aproximar mejor el gradiente y limitar el número de cálculos.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_iter = num_iter0
lr = lr0
m = m0

# Inicializar el array para la pérdida en cada iteración.
loss_sgd = np.zeros(num_iter)

for i in range(num_iter):
    # Seleccionar aleatoriamente un punto de datos de entrenamiento.
    k = np.random.randint(0, len(y))

    # Calcular el gradiente usando un solo punto de datos.
    g = -2 * x[k] * (y[k] - m * x[k])

    # Actualizar el parámetro, m.
    m = m - lr * g

    # Calcular la pérdida para el valor actualizado de m.
    e = y - m * x
    loss_sgd[i] = np.sum(e * e)

m_best = m

print('Minimum loss:   ', loss_sgd[-1])
print('Best parameter: ', m_best)

# Graficar la pérdida vs iteraciones.
plt.figure()
plt.plot(loss_sgd, 'c-')
plt.xlim(0, num_iter)
plt.ylim(0, np.max(loss_sgd))
plt.ylabel('Loss')
plt.xlabel('Iterations')
plt.title('Stochastic Gradient Descent')
plt.show()

plot_linear_model(x, y, m_best)

#### <font style="color: rgb(50,120, 229);"> 4.3 Descenso del Gradiente Estocástico en Mini-Batches </font>

En la sección anterior, vimos que es posible calcular el gradiente basado en un solo punto de datos aleatorio elegido en cada iteración. Siempre y cuando ejecutemos suficientes iteraciones, el Descenso del Gradiente Estocástico seguirá funcionando.

Sin embargo, usar más de un punto de datos para el cálculo del gradiente tiene dos ventajas:

1. Usar múltiples puntos de datos produce una estimación más precisa del gradiente.
2. Las GPUs son altamente eficientes en el procesamiento de cálculos de gradiente.

Por lo tanto, obtenemos mejores resultados y una convergencia más rápida si usamos un pequeño lote de puntos de datos, llamado **mini-lote**, para calcular los gradientes. Un enfoque de "mini-lote" encuentra un buen equilibrio entre usar todos los puntos de datos vs. solo un punto de datos.

Implementemos esto en código y veamos por nosotros mismos.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_iter = num_iter0
lr = lr0
m = m0

batch_size = 10

# Inicializar el array para la pérdida en cada iteración.
loss_sgd_mb = np.zeros(num_iter)

for i in range(num_iter):
    # Seleccionar aleatoriamente un batch de puntos de datos.
    k = np.random.randint(0, len(y), size=batch_size)

    # Calcular el gradiente usando el mini-batch.
    g = -2 * np.sum(x[k] * (y[k] - m * x[k])) / batch_size

    # Actualizar el parámetro, m.
    m = m - lr * g

    # Calcular la pérdida para el valor actualizado de m.
    e = y - m * x
    loss_sgd_mb[i] = np.sum(e * e) / batch_size

m_best = m

print('Minimum loss:   ', loss_sgd_mb[-1])
print('Best parameter: ', m_best)

# Graficar la pérdida vs iteraciones.
plt.figure()
plt.plot(loss_sgd_mb, 'c-')
plt.xlim(0, num_iter)
plt.ylim(0, np.max(loss_sgd_mb))
plt.ylabel('Loss')
plt.xlabel('Iterations')
plt.title('Stochastic Gradient Descent with Mini-Batch')
plt.show()

plot_linear_model(x, y, m_best)